In [83]:
import cv2
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt
import time

# Using mediapipe holistic, full body pose estimation

In [84]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [85]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [86]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [87]:
def draw_custom_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,  mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(255, 255, 204), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(225, 225, 225), thickness=1, circle_radius=1))
    
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(204, 153, 255), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=2))
    
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0, 128, 0), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))
    

# Test pose estimation and landmarks drawing

In [88]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
         
        draw_custom_landmarks(image, results)
        
        cv2.imshow("Camera", image)
        
        if cv2.waitKey(10) & 0XFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()
    

# Get landmarks keypoints from frames

In [89]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
  
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3) 

    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])

# Create keypoints data folder

In [90]:
import json

In [91]:
with open('C:/Users/wailb/Downloads/Compressed/archive_2/metadata.json', 'r') as json_file:
    metadata = json.load(json_file)

In [92]:
DATA_PATH = os.path.join('Keypoints_Data')

words = np.array([entry['gloss'] for entry in metadata])

In [93]:
missing_file_path = "C:/Users/wailb/Downloads/Compressed/archive_2/missing.txt" 

missing_IDs = []

with open(missing_file_path, 'r') as file:
    for line in file:
        missing_IDs.append(line.strip())

In [60]:
all_seq = []
i = 0
for entry in metadata:
        
    no_sequences = 0
    sequence_length =  []
    
    for instance in entry["instances"]:
        
        path = "C:/Users/wailb/Downloads/Compressed/archive_2/videos/"
        
        if instance["video_id"] not in missing_IDs:
            no_sequences += 1
            path = path + instance["video_id"] + '.mp4'
            sequence_length.append((instance["video_id"], instance["frame_start"], instance["frame_end"]))
            
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, entry['gloss'], sequence_length[sequence][0]))
        except:
            pass
    all_seq.append((entry['gloss'], no_sequences, sequence_length))
    i += 1
    print(i)
    print(entry['gloss'], '  /  ',no_sequences)


1
book   /   34
2
drink   /   24
3
computer   /   22
4
before   /   23
5
chair   /   22
6
go   /   19
7
clothes   /   23
8
who   /   19
9
candy   /   19
10
cousin   /   18
11
deaf   /   18
12
fine   /   17
13
help   /   16
14
no   /   18
15
thin   /   19
16
walk   /   16
17
year   /   16
18
yes   /   19
19
all   /   14
20
black   /   17
21
cool   /   18
22
finish   /   17
23
hot   /   18
24
like   /   19
25
many   /   16
26
mother   /   19
27
now   /   17
28
orange   /   18
29
table   /   17
30
thanksgiving   /   17
31
what   /   15
32
woman   /   17
33
bed   /   17
34
blue   /   14
35
bowling   /   17
36
can   /   13
37
dog   /   16
38
family   /   16
39
fish   /   16
40
graduate   /   16
41
hat   /   15
42
hearing   /   15
43
kiss   /   16
44
language   /   15
45
later   /   16
46
man   /   16
47
shirt   /   17
48
study   /   17
49
tall   /   17
50
white   /   17
51
wrong   /   17
52
accident   /   16
53
apple   /   15
54
bird   /   16
55
change   /   16
56
color   /   16
57
corn   /

456
stubborn   /   11
457
sunset   /   11
458
surgery   /   11
459
there   /   10
460
tiger   /   11
461
toast   /   12
462
toilet   /   9
463
tomorrow   /   12
464
town   /   12
465
transfer   /   12
466
tree   /   9
467
truck   /   11
468
uncle   /   10
469
vacation   /   10
470
weather   /   12
471
weekend   /   12
472
accept   /   10
473
adult   /   9
474
after   /   9
475
ago   /   10
476
allow   /   10
477
america   /   9
478
angel   /   9
479
answer   /   10
480
any   /   10
481
area   /   10
482
art   /   9
483
asl   /   8
484
aunt   /   9
485
awful   /   10
486
awkward   /   9
487
bacon   /   10
488
bark   /   11
489
bedroom   /   9
490
beer   /   10
491
belt   /   10
492
big   /   10
493
bitter   /   11
494
both   /   10
495
cake   /   10
496
california   /   11
497
canada   /   10
498
carrot   /   10
499
cause   /   10
500
challenge   /   10
501
cheap   /   10
502
chocolate   /   10
503
clean   /   9
504
cloud   /   9
505
compare   /   11
506
complex   /   10
507
contact   /

890
gloves   /   9
891
goat   /   8
892
goodbye   /   6
893
grow   /   7
894
gum   /   9
895
half   /   9
896
hawaii   /   8
897
herself   /   8
898
highway   /   9
899
homework   /   8
900
honor   /   8
901
ice cream   /   9
902
if   /   9
903
increase   /   8
904
india   /   8
905
information   /   9
906
interpret   /   8
907
interview   /   9
908
invite   /   7
909
israel   /   9
910
jesus   /   9
911
keep   /   9
912
keyboard   /   8
913
land   /   9
914
lawyer   /   8
915
lead   /   9
916
lesbian   /   9
917
line   /   8
918
lock   /   8
919
loud   /   9
920
lousy   /   9
921
lunch   /   9
922
machine   /   8
923
mad   /   8
924
memorize   /   9
925
minus   /   9
926
monday   /   8
927
mountain   /   8
928
mouth   /   8
929
mushroom   /   9
930
napkin   /   8
931
neck   /   9
932
next   /   9
933
ocean   /   8
934
open   /   9
935
overwhelm   /   10
936
owe   /   8
937
page   /   8
938
pain   /   8
939
parade   /   8
940
parents   /   9
941
perfect   /   8
942
period   /   8
943
p

1309
article   /   6
1310
assistant   /   6
1311
attend   /   6
1312
auction   /   6
1313
authority   /   6
1314
average   /   5
1315
awake   /   6
1316
b   /   7
1317
babysitter   /   8
1318
battle   /   6
1319
beginning   /   8
1320
belief   /   8
1321
bible   /   7
1322
bike   /   6
1323
blend   /   7
1324
bone   /   6
1325
bra   /   6
1326
brief   /   7
1327
broke   /   7
1328
bug   /   6
1329
bully   /   7
1330
button   /   7
1331
cabbage   /   6
1332
cabinet   /   7
1333
calculate   /   7
1334
calculator   /   7
1335
calculus   /   7
1336
camping   /   7
1337
canoe   /   6
1338
careless   /   6
1339
ceiling   /   8
1340
cemetery   /   7
1341
chapter   /   7
1342
choir   /   7
1343
circle   /   6
1344
come   /   7
1345
come here   /   8
1346
company   /   7
1347
complete   /   6
1348
concept   /   6
1349
concern   /   8
1350
confront   /   7
1351
confused   /   7
1352
congress   /   8
1353
connect   /   7
1354
conquer   /   6
1355
constitution   /   7
1356
contract   /   7
1357
co

1749
fire   /   5
1750
firefighter   /   6
1751
five   /   6
1752
flatter   /   6
1753
fool   /   5
1754
forgive   /   6
1755
former   /   6
1756
freeway   /   6
1757
from now on   /   6
1758
g   /   6
1759
gang   /   6
1760
gay   /   6
1761
geometry   /   6
1762
get up   /   6
1763
gift   /   6
1764
grab   /   6
1765
graduation   /   6
1766
grateful   /   7
1767
grey   /   5
1768
gymnastics   /   5
1769
h   /   6
1770
hang up   /   3
1771
hanukkah   /   5
1772
heap   /   6
1773
heart attack   /   6
1774
hill   /   6
1775
holy   /   6
1776
hop   /   6
1777
host   /   6
1778
hot dog   /   5
1779
human   /   6
1780
i   /   6
1781
ill   /   6
1782
illegal   /   6
1783
impact   /   5
1784
individual   /   6
1785
inspect   /   6
1786
inspire   /   6
1787
intersection   /   6
1788
ireland   /   7
1789
iron   /   6
1790
j   /   6
1791
jewelry   /   7
1792
journey   /   6
1793
joy   /   6
1794
july   /   6
1795
k   /   6
1796
kid   /   6
1797
kindergarten   /   6
1798
lady   /   6
1799
lamp   

In [94]:
all_seq[0]

('book',
 34,
 [('69241', 1, -1),
  ('68011', 1, -1),
  ('68012', 1, -1),
  ('70212', 2150, 2249),
  ('70266', 3732, 3852),
  ('07085', 1, -1),
  ('07086', 1, -1),
  ('07087', 1, -1),
  ('07069', 1, -1),
  ('07088', 1, -1),
  ('07089', 1, -1),
  ('07090', 1, -1),
  ('07091', 1, -1),
  ('07092', 1, -1),
  ('07093', 1, -1),
  ('07068', 1, -1),
  ('07094', 1, -1),
  ('07095', 1, -1),
  ('07096', 1, -1),
  ('07097', 1, -1),
  ('07070', 1, -1),
  ('07098', 1, -1),
  ('07099', 1, -1),
  ('07074', 1, -1),
  ('07075', 1, -1),
  ('07076', 1, -1),
  ('07077', 1, -1),
  ('07078', 1, -1),
  ('07079', 1, -1),
  ('07080', 1, -1),
  ('07081', 1, -1),
  ('07082', 1, -1),
  ('07083', 1, -1),
  ('07084', 1, -1)])

('45252', 7221)

1481

'Keypoints_Data'

In [133]:
def read_video_frame_range(video_path, start_frame, end_frame, action,video_id):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # Set starting frame position

    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        
        i = 0
        for _ in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)

            draw_custom_landmarks(image, results)
            
          
            keypoints = get_keypoints(results)
            npy_path = os.path.join(DATA_PATH, action, video_id, str(i))
            np.save(npy_path, keypoints)
            
            i += 1
    cap.release()
    cv2.destroyAllWindows()

In [135]:
i = 0
for action in all_seq:
    for video in action[2]:
        video_path = "C:/Users/wailb/Downloads/Compressed/archive_2/videos/" + video[0] +'.mp4'
        read_video_frame_range(video_path, video[1], video[2], action[0], video[0])
        video_path = ""
    i += 1
    print(i, ' | ', action[0])

1  |  book
2  |  drink
3  |  computer
4  |  before
5  |  chair
6  |  go
7  |  clothes
8  |  who
9  |  candy
10  |  cousin
11  |  deaf
12  |  fine
13  |  help
14  |  no
15  |  thin
16  |  walk
17  |  year
18  |  yes
19  |  all
20  |  black
21  |  cool
22  |  finish
23  |  hot
24  |  like
25  |  many
26  |  mother
27  |  now
28  |  orange
29  |  table
30  |  thanksgiving
31  |  what
32  |  woman
33  |  bed
34  |  blue
35  |  bowling
36  |  can
37  |  dog
38  |  family
39  |  fish
40  |  graduate
41  |  hat
42  |  hearing
43  |  kiss
44  |  language
45  |  later
46  |  man
47  |  shirt
48  |  study
49  |  tall
50  |  white
51  |  wrong
52  |  accident
53  |  apple
54  |  bird
55  |  change
56  |  color
57  |  corn
58  |  cow
59  |  dance
60  |  dark
61  |  doctor
62  |  eat
63  |  enjoy
64  |  forget
65  |  give
66  |  last
67  |  meet
68  |  pink
69  |  pizza
70  |  play
71  |  school
72  |  secretary
73  |  short
74  |  time
75  |  want
76  |  work
77  |  africa
78  |  basketball
79  | 

577  |  present
578  |  price
579  |  prison
580  |  push
581  |  raccoon
582  |  really
583  |  reason
584  |  religion
585  |  respect
586  |  rule
587  |  salad
588  |  schedule
589  |  sell
590  |  serious
591  |  shower
592  |  single
593  |  smart
594  |  smile
595  |  soft
596  |  sound
597  |  soup
598  |  spain
599  |  spray
600  |  squirrel
601  |  stomach
602  |  story
603  |  strange
604  |  street
605  |  stress
606  |  strict
607  |  strong
608  |  sugar
609  |  summer
610  |  suspect
611  |  sweetheart
612  |  tease
613  |  that
614  |  themselves
615  |  therapy
616  |  thirsty
617  |  ticket
618  |  top
619  |  tuesday
620  |  turkey
621  |  university
622  |  until
623  |  watch
624  |  weak
625  |  wedding
626  |  will
627  |  wind
628  |  world
629  |  worry
630  |  wow
631  |  young
632  |  add
633  |  airplane
634  |  already
635  |  also
636  |  analyze
637  |  and
638  |  angry
639  |  appear
640  |  arm
641  |  army
642  |  arrest
643  |  asia
644  |  ask
645  

1114  |  flexible
1115  |  flirt
1116  |  flood
1117  |  fork
1118  |  four
1119  |  fun
1120  |  funeral
1121  |  furniture
1122  |  gallaudet
1123  |  general
1124  |  ghost
1125  |  give up
1126  |  glass
1127  |  gorilla
1128  |  gray
1129  |  guide
1130  |  habit
1131  |  health
1132  |  hearing aid
1133  |  heaven
1134  |  heavy
1135  |  helmet
1136  |  hide
1137  |  high school
1138  |  hockey
1139  |  hold
1140  |  honest
1141  |  hug
1142  |  hunt
1143  |  image
1144  |  influence
1145  |  interesting
1146  |  interpreter
1147  |  iran
1148  |  italy
1149  |  jewish
1150  |  judge
1151  |  key
1152  |  label
1153  |  leaf
1154  |  left
1155  |  lend
1156  |  less
1157  |  linguistics
1158  |  lonely
1159  |  long
1160  |  magic
1161  |  mainstream
1162  |  major
1163  |  manager
1164  |  maximum
1165  |  me
1166  |  mexico
1167  |  middle
1168  |  mind
1169  |  misunderstand
1170  |  monster
1171  |  multiply
1172  |  myself
1173  |  nothing
1174  |  october
1175  |  odd
1176 

1613  |  amputate
1614  |  anatomy
1615  |  annoy
1616  |  anyway
1617  |  approach
1618  |  arizona
1619  |  assist
1620  |  assume
1621  |  attorney
1622  |  audiologist
1623  |  audiology
1624  |  austria
1625  |  author
1626  |  available
1627  |  award
1628  |  aware
1629  |  bank
1630  |  baptize
1631  |  basic
1632  |  battery
1633  |  berry
1634  |  beside
1635  |  between
1636  |  binoculars
1637  |  blow
1638  |  boast
1639  |  boil
1640  |  bookshelf
1641  |  bookstore
1642  |  boxing
1643  |  braid
1644  |  brain
1645  |  breakdown
1646  |  breakfast
1647  |  breeze
1648  |  bribe
1649  |  brochure
1650  |  buffalo
1651  |  burp
1652  |  bush
1653  |  bye
1654  |  camel
1655  |  candidate
1656  |  cannot
1657  |  captain
1658  |  carnival
1659  |  caterpillar
1660  |  cheerleader
1661  |  chemical
1662  |  chicago
1663  |  choke
1664  |  christ
1665  |  click
1666  |  closet
1667  |  clueless
1668  |  clumsy
1669  |  collect
1670  |  comma
1671  |  comment
1672  |  commit
1

'12306'

In [136]:
video

('63190', 1, -1)